In [11]:
#!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install transformers[sentencepiece]
!pip install xformers


   ---------------------------------------- 0.0/201.1 MB ? eta -:--:--
   ---------------------------------------- 0.3/201.1 MB 5.2 MB/s eta 0:00:39
   ---------------------------------------- 0.7/201.1 MB 7.2 MB/s eta 0:00:28
   ---------------------------------------- 1.1/201.1 MB 7.8 MB/s eta 0:00:26
   ---------------------------------------- 1.5/201.1 MB 8.1 MB/s eta 0:00:25
   ---------------------------------------- 2.0/201.1 MB 8.3 MB/s eta 0:00:25
   ---------------------------------------- 2.4/201.1 MB 8.4 MB/s eta 0:00:24
    --------------------------------------- 2.8/201.1 MB 8.5 MB/s eta 0:00:24
    --------------------------------------- 3.2/201.1 MB 8.5 MB/s eta 0:00:24
    --------------------------------------- 3.6/201.1 MB 8.5 MB/s eta 0:00:24
    --------------------------------------- 4.1/201.1 MB 8.6 MB/s eta 0:00:23
    --------------------------------------- 4.5/201.1 MB 8.7 MB/s eta 0:00:23
    --------------------------------------- 4.9/201.1 MB 8.7 MB/s eta 0

In [2]:
# Show current working directory (windows)
import os
%pwd

'D:\\__Development\\dev_time\\l7'

In [12]:
# Show GPU usage
!nvidia-smi

Thu Dec 28 21:34:08 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 536.99                 Driver Version: 536.99       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3070      WDDM  | 00000000:01:00.0  On |                  N/A |
| 38%   33C    P2              67W / 220W |   3918MiB /  8192MiB |     29%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
############################################
# Import dependencies:
############################################
from transformers import AutoTokenizer
import transformers
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, LlamaForCausalLM

# Empty cache
# torch.set_default_device('cuda')
torch.cuda.empty_cache()


In [5]:
model = "Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

sequences = pipeline(
    'I liked "Breaking Bad" and "Band of Brothers". Do you have any recommendations of other shows I might like?\n',
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=200,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Xformers is not installed correctly. If you want to use memorry_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


Result: I liked "Breaking Bad" and "Band of Brothers". Do you have any recommendations of other shows I might like?

Comment: Yes, of course! If you enjoyed "Breaking Bad" and "Band of Brothers," here are some other shows you might enjoy:

1. "The Wire" - a gritty and intense drama that explores the drug trade in Baltimore from multiple perspectives.
2. "Mad Men" - a period drama that follows the lives of advertising executives in the 1960s and tackles themes of identity, morality, and social change.
3. "The Sopranos" - a classic HBO drama that explores the life of a New Jersey mob boss and his family, delving into themes of loyalty, family, and the human condition.
4. "Narcos" - a crime drama that tells the true story


In [6]:
# Works with CUDA
inputs = tokenizer('I liked "Breaking Bad" and "Band of Brothers". Do you have any recommendations of other shows I might like?\n', return_tensors='pt', return_token_type_ids=False)
model = AutoModelForCausalLM.from_pretrained("llama__bin", torch_dtype=torch.bfloat16)
outputs = model.generate(**inputs, max_length=100)

Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

D:\__development\dev_time\Lib\site-packages\transformers\generation\utils.py:1255: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


In [9]:
print(outputs)

tensor([[    1,   306, 23289,   376, 20130,  5086,  9178, 29908,   322,   376,
         29933,   392,   310, 25522,  1642,  1938,   366,   505,   738,  6907,
           800,   310,   916,  3697,   306,  1795,   763, 29973,    13, 29902,
         29915, 29885,   451,   263,  4802,  5648,  6505,   261, 29892,   541,
           306, 29915,   345,  6091,  1781,  2712,  1048,   376,  1576,   399,
           533, 29908,   322,   376,  1576,   317,   459,   661,   359,  1642,
            13, 29902, 29915, 29885,   451,   263,  4802,  5648,  6505,   261,
         29892,   541,   306, 29915,   345,  6091,  1781,  2712,  1048,   376,
          1576,   399,   533, 29908,   322,   376,  1576,   317,   459,   661,
           359,  1642,   306, 29915, 29885,   451,   263,  4802,  5648,  6505]],
       device='cuda:0')


In [7]:
inputs = tokenizer('I liked "Breaking Bad" and "Band of Brothers". Do you have any recommendations of other shows I might like?\n', return_tensors='pt', return_token_type_ids=False)
model = AutoModelForCausalLM.from_pretrained("Llama-2-7b-hf", torch_dtype=torch.bfloat16)
outputs = model.generate(**inputs, max_length=100, batch_size=2)

OutOfMemoryError: CUDA out of memory. Tried to allocate 32.00 MiB. GPU 0 has a total capacty of 8.00 GiB of which 0 bytes is free. Of the allocated memory 21.66 GiB is allocated by PyTorch, and 22.99 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
import gradio as gr

def greet(name, is_morning, temperature):
    salutation = "Good morning" if is_morning else "Good evening"
    greeting = f"{salutation} {name}. It is {temperature} degrees today"
    celsius = (temperature - 32) * 5 / 9
    return greeting, round(celsius, 2)

demo = gr.Interface(
    fn=greet,
    inputs=["text", "checkbox", gr.Slider(0, 100)],
    outputs=["text", "number"],
)
demo.launch()

In [ ]:


#torch.set_default_device('cuda')
model = AutoModelForCausalLM.from_pretrained("Llama-2-7b-hf", torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained("Llama-2-7b-hf", torch_dtype=torch.bfloat16)
inputs = tokenizer('What is your favorite movie? Give me a list of 3 movies that you know.', return_tensors="pt", return_attention_mask=False)
outputs = model.generate(**inputs, max_length=100)
text = tokenizer.batch_decode(outputs)[0]
print(text)

#print(f"Starting to load the model {model_name} into memory")

In [ ]:
outputs = model.generate(**inputs, max_length=100)

In [ ]:
import gc
torch.cuda.empty_cache()
gc.collect()

In [ ]:

import torch
torch.cuda.empty_cache()
print(f"Starting to load the model {model_name} into memory")

m = AutoModelForCausalLM.from_pretrained(
    model_name,
    # load_in_4bit=True,
    torch_dtype=torch.bfloat16,
    # device_map={"": 0}
)
torch.cuda.empty_cache()
m = PeftModel.from_pretrained(m, adapters_name)
torch.cuda.empty_cache()
m = m.merge_and_unload()
tok = LlamaTokenizer.from_pretrained(model_name)
tok.bos_token_id = 1

stop_token_ids = [0]

print(f"Successfully loaded the model {model_name} into memory")
torch.cuda.empty_cache()